# Galaxy Tracker

In [6]:
from matplotlib import pyplot as plt
import pandas as pd
from auriga.images import add_redshift, figure_setup, set_axs_configuration
from auriga.paths import Paths
from auriga.settings import Settings
from auriga.parser import parse

In [2]:
figure_setup()

In [3]:
def add_panel_for_galaxy(simulation: str, ax: plt.Axes):
    """
    This method adds a panel with the results for a given simulation.

    Parameters
    ----------
    simulation : str
        The simulation.
    ax : plt.Axes
        The ax to which to add the plot.
    """

    galaxy, rerun, resolution = parse(simulation=simulation)
    paths = Paths(galaxy, rerun, resolution)
    df = pd.read_csv(f"../{paths.results}temporal_data.csv")
    ax.plot(df["Time_Gyr"], df["MainHaloIdx"],
            c='tab:red', lw=1.0, label='Halo', zorder=10)
    ax.plot(df["Time_Gyr"], df["MainSubhaloIdx"],
            c='tab:green', lw=1.0, label='Subhalo', zorder=11)
    ax.text(x=0.95, y=0.95, size=6.0,
            s=r"$\texttt{" + f"au{galaxy}_or_l4".upper() + "}$",
            ha='right', va='top', transform=ax.transAxes)

In [9]:
def plot_gal_tracker_for_sample(simulations: list,
                                filename: str):
    """
    This method creates a plot of the index of the main halo/subhalo
    a function of cosmic time for each simulation.

    Parameters
    ----------
    simulations : str
        A list of simulations to plot.
    filename : str
        The name of the output file.
    """

    n_simulations = len(simulations)

    fig = plt.figure(figsize=(7.2, 7.2))
    gs = fig.add_gridspec(nrows=6, ncols=5, hspace=0.0, wspace=0.0)
    axs = gs.subplots(sharex=True, sharey=True)

    set_axs_configuration(
        xlim=(0, 14), ylim=(-0.5, 5.5),
        xticks=[2, 4, 6, 8, 10, 12, 14], yticks=[0, 1, 2, 3, 4, 5],
        xlabel="Time [Gyr]", ylabel="Index",
        axs=axs, n_used=n_simulations)

    for idx, ax in enumerate(axs.flat):
        ax.grid(True, ls='-', lw=0.25, c='silver')
        if ax.get_subplotspec().is_first_row():
            add_redshift(ax)
        if idx < n_simulations:
            add_panel_for_galaxy(simulations[idx], ax=ax)
        else:
            ax.axis("off")

    for ext in ["pdf", "png"]:
        fig.savefig(f"../images/galaxy_tracker/{filename}.{ext}")
    plt.close(fig)

In [10]:
# Create plots for all galaxies in two figures
settings = Settings()
originals = [f"au{i}_or_l4" for i in settings.galaxies]
reruns = [f"au{i}_re_l4" for i in settings.reruns]
plot_gal_tracker_for_sample(simulations=originals, filename="originals")
plot_gal_tracker_for_sample(simulations=reruns, filename="reruns")